In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simps
from scipy.optimize import curve_fit

In [ ]:
cd ./As_intensity

In [4]:
def resort(t,f):
    t = np.array(t)
    f = np.array(f)
    array = np.column_stack([t,f])
    new_array= array[np.argsort(array[:,0])]
    new_t = new_array[:,0]
    new_f = new_array[:,1]
    return new_t, new_f

In [6]:
data = np.loadtxt('limb_atlkurab_vt1_K0_hydromol_means.txt', skiprows=2)

In [7]:
wavelength = data[:,0]

wavelength_m = []
for i in wavelength:
    new = i*1e-9
    wavelength_m.append(new)
    
freq = []
for i in wavelength_m:
    new = (3e8)/i
    freq.append(new)

In [8]:
intensity_si = []
for i in range(1,10):
    intensity1_cgs = data[:,i]
    intensity1_si = []
    for j in intensity1_cgs:
        new = (j*1e-7)*(100**2)
        intensity1_si.append(new)
    intensity_si.append(intensity1_si)

In [9]:
wav_lower = 350e-9 #set lower limit on wavelength
wav_higher = 3000e-9 #set upper limit on wavelength

val_low = []
val_high = []
for i in range(0,len(wavelength_m)):
    new_low = abs(wav_lower - wavelength_m[i])
    new_high = abs(wav_higher - wavelength_m[i])
    val_low.append(new_low)
    val_high.append(new_high)
indx_low = val_low.index(min(val_low))
indx_high = val_high.index(min(val_high))
print(wavelength_m[indx_low])
print(wavelength_m[indx_high])

3.49e-07
2.995e-06


In [10]:
def model_func(x, a, b):
    mu = np.sqrt(1-(x/np.pi))
    return area_total[0] * (1-(a*(1-(mu**b))))

    #area_total[0] * (1-(a*(1-(mu**b))))

wavs = []
I0s = []
a_bests = []
b_bests = []
wavelength_m=list(wavelength_m)
for i in range(indx_low,indx_high+1):
    if i%30 == 0:
        delta_freq = ((-(3e8/(wavelength_m[i+1]))+(3e8/(wavelength_m[i])))/2) + ((-(3e8/(wavelength_m[i]))+(3e8/(wavelength_m[i-1])))/2)
    
        I = []
        for j in range(0,9):
            new = intensity_si[j][i] * delta_freq
            I.append(new)
        I = np.asarray(I)
    
        area_total = I
        costheta = [1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2]
        ster=[]
        for k in costheta:
            new = (np.pi)*(1-(k**2))
            ster.append(new)
    
        xData = np.asarray(ster)
        yData = np.asarray(area_total)

        sigma = np.ones(len(xData))

        popt, pcov = curve_fit(model_func, xData, yData, p0=(0.67595786, 0.7330585), sigma=sigma, maxfev=10000)

        a_best = popt[0]
        b_best = popt[1]
    
        wavs.append(wavelength_m[i])
        I0s.append(area_total[0])
        a_bests.append(a_best)
        b_bests.append(b_best)
    
    v = np.column_stack((wavs,I0s,a_bests,b_bests))
    np.savetxt('contrasttest2.txt', v)   